# Homework 2: Network Optimization and Non-linear Models

## 1. Part A: Network Optimization (Discrete Model)

**Topic:** **Optimization of the BiciMad Network Capacity (Optimal Slot Assignment)**

The objective is to minimize the total network cost, which includes the cost of capacity expansion and the penalty cost for user rejection (when there is no bike or no free dock), making the capacity ($\mathbf{S_i}$) a discrete variable. [Image of a network optimization model diagram showing facilities and customer zones with flow and capacity constraints]

### Parameters (Input Data)

| Parameter | Description | Source Value |
| :--- | :--- | :--- |
| $N$ | Set of all BiciMad stations. | (Network nodes, over 600). |
| $D_{ij}$ | Total number of trips from station $i$ to station $j$. | `matriz_demanda_Dij.csv` |
| $c_{ij}$ | Unit cost of movement/transport from $i$ to $j$. | `matriz_costos_Cij.csv` |
| $C^{\text{initial}}_i$ | Initial capacity of slots at station $i$. | `bikestationbicimad_csv.csv` |
| $C^{\text{slot}}$ | Cost to install a new dock/slot. | **Assumed**, e.g., $\mathbf{C^{\text{slot}} = €500}$ per slot. |
| $P^{\text{rejection}}$ | Penalty for user rejection. | **Assumed**, e.g., $\mathbf{P^{\text{rejection}} = €5}$ per rejection. |
| $B$ | Maximum budget for capacity expansion. | **Assumed**, e.g., $\mathbf{B = €100,000}$. |

### Decision Variables

* $\mathbf{S_i \in \mathbb{Z}^+}$: Final number of docks (slots) assigned to station $i$. (**Discrete/Integer** Variable).

### Mathematical Formulation

**Objective (Cost Minimization):**

Minimize the Capacity Expansion Cost plus the Rejection Penalty.
$$\min \sum_{i \in N} C^{\text{slot}} \cdot \max(0, S_i - C^{\text{initial}}_i) + \sum_{i \in N} P^{\text{rejection}} \cdot (\text{Outbound Rejection}_i + \text{Inbound Rejection}_i)$$

**Key Constraints (Subject to):**

1.  **Capacity Limit (Variable Domain):** The number of slots must be within a realistic operational range.
    $$\mathbf{S_{min} \leq S_i \leq S_{max}}, \quad \forall i \in N \quad \text{(e.g., } 10 \leq S_i \leq 50 \text{)}$$
2.  **Budget (If considered as a constraint):**
    $$\sum_{i \in N} C^{\text{slot}} \cdot \max(0, S_i - C^{\text{initial}}_i) \leq B$$
3.  **Loss Constraints (e.g., Inbound Rejection):** This constraint links the final capacity to the demand for arrival slots.
    $$\text{Inbound Rejection}_i = \max(0, \sum_{j} D_{ji} - S_i)$$

---


In [14]:
import pandas as pd
from pulp import *
import numpy as np

# --- 1. CARGAR DATOS GENERADOS ---
# Matriz de Demanda (Flujo Total: Origen i -> Destino j)
matriz_demanda_Dij = pd.read_csv('matriz_demanda_Dij.csv', index_col=0)

# Datos de Nodos (ID, Lat/Long, Capacidad Inicial)
df_stations = pd.read_csv('bikestationbicimad_csv.csv', delimiter=';')
df_stations['number'] = pd.to_numeric(df_stations['number'], errors='coerce')
df_stations = df_stations.dropna(subset=['number'])
df_stations['ID_Estacion'] = df_stations['number'].astype(int)
df_initial_capacity = df_stations[['ID_Estacion', 'TotalBases']].set_index('ID_Estacion')

# --- 2. DEFINIR CONJUNTOS Y PARÁMETROS FIJOS ---
N = matriz_demanda_Dij.index.tolist()  # Conjunto de todas las estaciones
C_inicial = df_initial_capacity['TotalBases'].to_dict()

# --- 3. PARÁMETROS ASUMIDOS (Costos/Límites del Modelo) ---
C_slot = 500         # Costo de instalar un nuevo slot (€/slot)
P_rechazo_out = 5.0  # Penalización por rechazo de salida (No hay bici) (€/viaje)
P_rechazo_in = 5.0   # Penalización por rechazo de llegada (No hay slot) (€/viaje)
B_max = 100000       # Presupuesto máximo para expansión (€)
S_min = 10           # Capacidad mínima de slots por estación
S_max = 50           # Capacidad máxima de slots por estación

In [2]:
# --- 4. CALCULAR DEMANDAS TOTALES (Necesario para las pérdidas) ---

# Demanda total de SALIDA de cada estación i (viajes Origen i -> Destino j)
# (Demanda de bicicletas)
D_out = matriz_demanda_Dij.sum(axis=1).to_dict()

# Demanda total de LLEGADA a cada estación i (viajes Origen j -> Destino i)
# (Demanda de slots)
D_in = matriz_demanda_Dij.sum(axis=0).to_dict()

# --- 5. DEFINICIÓN DEL PROBLEMA Y VARIABLES ---

prob = LpProblem("BiciMad_Capacity_Optimization", LpMinimize)

# S_i: Número final de slots asignados a la estación i (Variable Discreta/Entera)
S = LpVariable.dicts("Final_Slots", N, lowBound=S_min, upBound=S_max, cat=LpInteger)

# Variables auxiliares para modelar las pérdidas (Rechazos). Se usan variables Continuas
# para que PuLP pueda encontrar la solución, y luego se interpretan.
# R_in_i: Número de viajes que son rechazados por falta de slots en la llegada a i.
R_in = LpVariable.dicts("Rejection_Inbound", N, lowBound=0, cat=LpContinuous)

# R_out_i: Número de viajes que son rechazados por falta de bicis en la salida de i.
R_out = LpVariable.dicts("Rejection_Outbound", N, lowBound=0, cat=LpContinuous)

# E_i: Expansión de Capacidad (slots nuevos > 0)
E = LpVariable.dicts("Capacity_Expansion", N, lowBound=0, cat=LpContinuous)

In [3]:
# --- 6. FUNCIÓN OBJETIVO ---
# Min C_slot * SUM(Expansión) + P_rechazo * SUM(Rechazos)

prob += (
    # Costo 1: Expansión de Capacidad
    lpSum([C_slot * E[i] for i in N]) + 
    
    # Costo 2: Penalización por Rechazo de Llegada (Falta de slot)
    lpSum([P_rechazo_in * R_in[i] for i in N]) +
    
    # Costo 3: Penalización por Rechazo de Salida (Falta de bici)
    lpSum([P_rechazo_out * R_out[i] for i in N])
)

# --- 7. RESTRICCIONES CLAVE ---

# Restricción 1: Definición de la Expansión (Solo cuenta si S_i > C_inicial_i)
for i in N:
    # Usamos .get(i, 0) para manejar posibles claves faltantes en C_inicial
    prob += E[i] >= S[i] - C_inicial.get(i, 0), f"Expansion_Def_{i}"

# Restricción 2: Definición del Rechazo de Llegada (No hay slot)
# El rechazo es el exceso de demanda de llegada sobre la capacidad final S_i
for i in N:
    # **CORRECCIÓN CLAVE:** Usamos .get(i, 0) para manejar estaciones sin llegadas
    prob += R_in[i] >= D_in.get(i, 0) - S[i], f"Rejection_Inbound_Def_{i}"

# Restricción 3: Definición del Rechazo de Salida (No hay bici)
# Asumimos que el rechazo de salida se debe a que la demanda supera la capacidad inicial (stock inicial).
for i in N:
    # Usamos .get(i, 0) para manejar estaciones sin salidas o sin capacidad inicial
    prob += R_out[i] >= D_out.get(i, 0) - C_inicial.get(i, 0), f"Rejection_Outbound_Def_{i}"
    
# Restricción 4 (Opcional): Presupuesto Máximo
prob += lpSum([C_slot * E[i] for i in N]) <= B_max, "Budget_Constraint"

In [5]:
# --- 8. RESOLVER EL PROBLEMA ---
prob.solve()

print("Estado de la Solución:", LpStatus[prob.status])
print("Costo Total Mínimo (Objetivo): €", round(value(prob.objective), 2))

# --- 9. INTERPRETACIÓN DE RESULTADOS ---
results = []
for i in N:
    # Solo incluir estaciones que tienen una capacidad inicial definida y una solución válida
    if i in C_inicial and value(S[i]) is not None:
        capacidad_optima = value(S[i])
        capacidad_inicial = C_inicial.get(i)
        
        results.append({
            'Estacion_ID': i,
            'Capacidad_Inicial': capacidad_inicial,
            'Capacidad_Optima': capacidad_optima,
            'Expansión_Necesaria': max(0, capacidad_optima - capacidad_inicial),
            'Rechazo_Llegada': round(value(R_in[i]), 2),
            'Rechazo_Salida': round(value(R_out[i]), 2)
        })

df_results = pd.DataFrame(results)

# Filtrar solo las estaciones que requieren una acción o tienen rechazo significativo
df_action = df_results[
    (df_results['Expansión_Necesaria'] > 0) | 
    (df_results['Rechazo_Llegada'] > 0.01) | 
    (df_results['Rechazo_Salida'] > 0.01)
]

print("\n--- Top 10 Estaciones con Cambios Óptimos o Rechazos ---")

# === LÍNEA CORREGIDA PARA EVITAR EL ERROR ===
# Usamos to_string() en lugar de to_markdown()
print(df_action.sort_values(by='Expansión_Necesaria', ascending=False).head(10).to_string(index=False))

expansion_cost = df_action['Expansión_Necesaria'].sum() * C_slot
print(f"\nCosto Total Real de Expansión de Capacidad: €{expansion_cost:.2f}")

Estado de la Solución: Optimal
Costo Total Mínimo (Objetivo): € 1344030.0

--- Top 10 Estaciones con Cambios Óptimos o Rechazos ---
 Estacion_ID  Capacidad_Inicial  Capacidad_Optima  Expansión_Necesaria  Rechazo_Llegada  Rechazo_Salida
           1                 23              10.0                    0              0.0           723.0
           2                 27              10.0                    0              0.0           494.0
           3                 19              10.0                    0              0.0          1154.0
           4                 27              10.0                    0              0.0           920.0
           5                 27              10.0                    0              0.0          1039.0
           6                 19              10.0                    0              0.0          1232.0
           7                 19              10.0                    0              0.0          1313.0
           8                 24     

### Different Scenario Analysis

#### Why this Analysis?

The initial execution of the linear model resulted in an optimal investment of **€0**, indicating that under the current parameter settings, the Cost of Infrastructure ($C_{slot}$) significantly outweighs the Penalty for Service Failure ($P_{penalty}$). Because this is a linear optimization model, the decision logic is mathematically **binary**: the solver simply calculates whether it is cheaper to build a slot or pay the penalty. To validate the robustness of our expansion plan, we must compare two opposing strategic realities.

* **Scenario A:**
    * **Parameters:** Cost = €500 | Penalty = €5 ($C_{slot} > P_{penalty}$)
    * **Logic:** This represents a financial strategy where budget protection is prioritized over service levels. Since the cost of construction is 100 times higher than the cost of rejecting a user, the mathematical optimum is to **freeze investment** and accept high rejection rates. In this scenario, it is simply "cheaper to pay the fine than to fix the problem."

* **Scenario B:**
    * **Parameters:** Cost = €500 | Penalty = €600 ($P_{penalty} > C_{slot}$)
    * **Logic:** This simulates a strategic shift where BiciMad prioritizes brand reputation. By setting the penalty higher than the construction cost, the model effectively understands that losing a recurring customer is more expensive than building a single slot. This forces the solver to utilize the entire budget to avoid rejections, leading to a massive expansion of the network.

In [42]:
from pulp import *

# We compare the two critical realities:
scenarios_A = {
    "A. Financial Focus": {
        "C_slot": 500,       
        "P_rechazo": 5      
    },
    "B. Service Focus": {
        "C_slot": 500,       
        "P_rechazo": 600 
    }
}


for name, params in scenarios_A.items():
    
    # 1. Initialize Problem
    prob_sen = LpProblem(f"Scenario_{name[:5]}", LpMinimize)
    c_slot = params["C_slot"]
    p_rechazo = params["P_rechazo"]
    
    # 2. Variables
    S_var = LpVariable.dicts("Slots_Final", N, lowBound=S_min, upBound=S_max, cat=LpInteger)
    E_var = LpVariable.dicts("Expansion", N, lowBound=0, cat=LpInteger)
    R_in = LpVariable.dicts("R_In", N, lowBound=0, cat=LpContinuous)
    R_out = LpVariable.dicts("R_Out", N, lowBound=0, cat=LpContinuous)
    
    # 3. Objective Function
    prob_sen += (
        lpSum([c_slot * E_var[i] for i in N]) + 
        lpSum([p_rechazo * (R_in[i] + R_out[i]) for i in N])
    )
    
    # 4. Constraints
    for i in N:
        # Link Expansion to Final Capacity
        prob_sen += S_var[i] == C_inicial.get(i, 0) + E_var[i]
        
        # KEY FIX: Expansion helps BOTH Inbound and Outbound
        # We assume expanding capacity helps satisfy demand in both directions
        prob_sen += R_in[i] >= D_in.get(i, 0) - S_var[i]
        prob_sen += R_out[i] >= D_out.get(i, 0) - S_var[i] 
        
    # Budget Constraint
    prob_sen += lpSum([c_slot * E_var[i] for i in N]) <= B_max

    # 5. Solve
    prob_sen.solve(PULP_CBC_CMD(msg=False))
    
    # 6. Metrics
    nuevos_slots = sum(value(E_var[i]) for i in N)
    inversion = nuevos_slots * c_slot
    
    # Detailed output to debug
    print(f"SCENARIO: {name}")
    print(f"  > Ratio: Cost (€{c_slot}) vs. Penalty (€{p_rechazo})")
    print(f"  > Investment Decision:  €{inversion:,.0f}")
    print(f"  > New Slots Built:      {int(nuevos_slots)}")
  
    
    print("-" * 60)

SCENARIO: A. Financial Focus
  > Ratio: Cost (€500) vs. Penalty (€5)
  > Investment Decision:  €0
  > New Slots Built:      0
------------------------------------------------------------
SCENARIO: B. Service Focus
  > Ratio: Cost (€500) vs. Penalty (€600)
  > Investment Decision:  €100,000
  > New Slots Built:      200
------------------------------------------------------------


### Interpretation of Results: 

In Scenario B, the model utilizes **100% of the available budget (€100,000)**. This behavior is a critical economic indicator of the network's status. The reasons for this total budget usage are the following:

#### 1. Positive Net Benefit
In Scenario B, the parameters create a net benefit for every new slot:
* **Cost to Build:** €500
* **Penalty Saved:** €600
* **Net Benefit per Slot:** +€100

Because the net benefit is positive, the solver determines that **building a slot is always strictly better than not building one**. Consequently, the instruction becomes: *"Build as many slots as physically and financially possible."*


#### 2. The Budget as the Binding Constraint
Since the model is incentivized to expand indefinitely to eliminate penalties, it will continue to add slots until it hits a limit. In this case, the **Budget Constraint ($B_{max}$)** acts as the bottleneck. The model stops investing at exactly €100,000 not because the network is fixed, but simply because it has run out of capital. This indicates that the budget constraint is **binding**.

#### 3. Massive Structural Deficit
The most important strategic insight from this result is the magnitude of the infrastructure gap.
* The fact that the budget is fully exhausted implies that **the cost to fix the network's total deficit is significantly higher than €100,000.**
* If the network only needed €50,000 to solve all rejection problems, the model would have spent €50,000 and stopped.
* Since it hit the €100,000 ceiling immediately, we can conclude that the BiciMad network is **severely underfunded** relative to the actual user demand, assuming a customer-centric service standard.

In [44]:

# We are going to analyze how different budget levels affect the solution


C_SLOT = 500
P_RECHAZO = 600 

# 2. Test different Budget levels (from €100k to €5 Million)
budget_levels = [100000, 200000, 500000, 1000000, 2000000, 5000000]

print(f"--- ANALYZING NETWORK SATURATION (Fixed Penalty: €{P_RECHAZO}) ---\n")

for b_lev in budget_levels:
    
    # Initialize
    prob = LpProblem(f"Budget_{b_lev}", LpMinimize)
    
    # Variables
    S_var = LpVariable.dicts("Slots", N, lowBound=S_min, upBound=S_max, cat=LpInteger)
    E_var = LpVariable.dicts("Expansion", N, lowBound=0, cat=LpInteger)
    R_in = LpVariable.dicts("R_In", N, lowBound=0, cat=LpContinuous)
    R_out = LpVariable.dicts("R_Out", N, lowBound=0, cat=LpContinuous)
    
    # Objective
    prob += (
        lpSum([C_SLOT * E_var[i] for i in N]) + 
        lpSum([P_RECHAZO * (R_in[i] + R_out[i]) for i in N])
    )
    
    # Constraints
    for i in N:
        prob += S_var[i] == C_inicial.get(i, 0) + E_var[i]
        prob += R_in[i] >= D_in.get(i, 0) - S_var[i]
        prob += R_out[i] >= D_out.get(i, 0) - S_var[i]
        
    # VARIABLE BUDGET CONSTRAINT
    prob += lpSum([C_SLOT * E_var[i] for i in N]) <= b_lev

    # Solve
    prob.solve(PULP_CBC_CMD(msg=False))
    
    # Metrics
    invested = sum(value(E_var[i]) for i in N) * C_SLOT
    nuevos_slots = sum(value(E_var[i]) for i in N)
    
    # Calculate % of Budget Used
    pct_used = (invested / b_lev) * 100
    
    print(f"Budget Available: €{b_lev:,.0f}")
    print(f"  > Amount Spent:   €{invested:,.0f} ({pct_used:.1f}%)")
    print(f"  > Slots Built:    {int(nuevos_slots)}")
    
    if pct_used < 99:
        print("  *** SATURATION REACHED: Network is fully optimized! ***")
    
    print("-" * 60)

--- ANALYZING NETWORK SATURATION (Fixed Penalty: €600) ---

Budget Available: €100,000
  > Amount Spent:   €100,000 (100.0%)
  > Slots Built:    200
------------------------------------------------------------
Budget Available: €200,000
  > Amount Spent:   €200,000 (100.0%)
  > Slots Built:    400
------------------------------------------------------------
Budget Available: €500,000
  > Amount Spent:   €500,000 (100.0%)
  > Slots Built:    1000
------------------------------------------------------------
Budget Available: €1,000,000
  > Amount Spent:   €1,000,000 (100.0%)
  > Slots Built:    2000
------------------------------------------------------------
Budget Available: €2,000,000
  > Amount Spent:   €2,000,000 (100.0%)
  > Slots Built:    4000
------------------------------------------------------------
Budget Available: €5,000,000
  > Amount Spent:   €3,227,500 (64.5%)
  > Slots Built:    6455
  *** SATURATION REACHED: Network is fully optimized! ***
----------------------------

### Budget Saturation & Structural Deficit Analysis

To determine the true scale of the network's infrastructure deficit, we performed a test by incrementally increasing the available budget ($B_{max}$) under a service-focused strategy (Penalty > Cost). The results reveal the financial requirement to fully optimize the BiciMad network.

#### 1. Complete Budget Usage (€100k – €2M)
For all budget levels up to **2 000 000€**, the model exhibits **100% Budget Utilization**. 
* This consumption indicates massive unfulfilled demand. 
* Even with a 20x increase in the current budget (from 100k€ to 2M€), the network needs more slots, proving that the infrastructure is severely undersized relative to user demand.


#### 2. The Saturation Point (€3,227,500)
The analysis identifies the precise **Saturation Point at 3 227 500€** (6,455 new slots).
* When provided with 5 000 000€, the model stops spending at approx. 3.2M€. 
* **Meaning:** At this investment level, every single station where expansion is physically possible ($S_{max}$) and economically viable has been expanded. Throwing more money at the problem beyond 3.2M€ yields no further benefit because the demand is fully covered.

#### 3. Strategic Conclusion: 
Comparing the saturation point to the current allocation reveals a critical gap:
* **Required Budget:** €3,227,500
* **Current Budget:** €100,000
* **Coverage:** **3.1%**

**Final Insight:** The current budget of €100,000 covers only **~3%** of the network's actual infrastructure needs. To transition from a "Financial Focus" to a "Service Excellence" standard, BiciMad would need to increase its capital allocation by a factor of **32x**. The current investment is not a solution, but merely a patch on a much larger structural problem.

### Interpretation of Results 

#### 1. Solution Status and Total Objective Cost
* **Solution Status: `Optimal`**
  The solver successfully found the best possible distribution of dock capacities to minimize the defined cost function while respecting all budget and operational constraints.
* **Minimum Total Cost: `€1,344,030.0`**
  This value represents the "System Cost," which is the sum of expansion investments and, primarily, the **penalty costs** for users who could not find a bike (Outbound Rejection) or a free dock (Inbound Rejection).

#### 2. Capacity Expansion and Budget Analysis
* **Real Expansion Cost: `€0.00`**
* **Necessary Expansion: `0 slots`**
  The model decided **not to expand any station**, leaving the €100,000 budget completely untouched.

**Why did this happen?**
If the cost of adding a new slot (€500) is higher than the marginal benefit of reducing user rejection at that specific point, the model will choose to pay the "rejection fine" instead of investing in infrastructure. 

#### 3. Top 10 Stations: Rejection vs. Capacity
By analyzing the top stations, a clear operational bottleneck is identified:
* **Capacity Reduction (`Capacidad_Optima` = 10):** The model reduced the capacity of most stations to the minimum allowed ($S_{min} = 10$). This suggests that having excess docks is currently not providing value to the network.
* **High Outbound Rejection (Lack of Bikes):** Stations like **ID 9** (1,802 rejections) and **ID 7** (1,313 rejections) show massive demand that cannot be met.
    * An **Outbound Rejection** occurs when a user wants a bike, but the station is empty.
    * Since Part A only optimizes **slots (infrastructure)** and not the movement of bikes, the model concludes that adding more docks is useless if there are no bikes to fill them.



#### 4. Strategic Conclusion
The "optimal" solution reveals that **physical infrastructure is not the current bottleneck of BiciMad**. The network is suffering from a severe **geographical imbalance**.

* **Operational Insight:** Even if we had an infinite budget for docks, users would still be rejected because the bikes are not where the demand is.
* **Transition to Part B:** This result perfectly justifies the need for the **Non-linear Model (Part B)**. While Part A shows that hardware expansion is not the answer, Part B will focus on **Logistics (Rebalancing)** to ensure the bikes are distributed efficiently throughout the day.


## 2. Part B: Non-linear Optimization

**Topic:** **Optimization of Bicycle Inventory at a Critical Station (Station 57)**

The objective is to minimize the total cost of the rebalancing operation, including the linear cost of moving bikes and the **non-linear** cost of losses due to user rejection (a fine which represents the user's frustration when the system fails). 

We have chosen Station 57 as we have seen that is the one with the highest number of arrivals and departures. We want to analyze the system's performance in a critical station rather than in the entire network simultaneously, which we considered non-viable.

### Parameters (Input Data)

| Parameter | Description | Source Value |
| :--- | :--- | :--- |
| $H$ | Set of hourly periods (0 to 23). | `parametros_operacionales_bicimad.csv` |
| $\lambda_h$ | Outbound rate of bikes (demand) in period $h$ (bikes/min). | `Flujo_Salida_por_Min` column. |
| $\mu_h$ | Inbound rate of bikes (supply) in period $h$ (bikes/min). | `Flujo_Llegada_por_Min` column. |
| $C_{\text{move}}$ | Cost to rebalance one bicycle for period $h

In [15]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize, LinearConstraint
import sys

# --- 1. CARGA DE DATOS Y DEFINICIÓN DE PARÁMETROS DEL PROBLEMA ---

df_operaciones = pd.read_csv('parametros_operacionales_bicimad.csv')

N_PERIODS = len(df_operaciones) # 24 períodos horarios

# Parámetros por hora (convertidos a arrays de numpy)
LAMBDA_H = df_operaciones['Flujo_Salida_por_Min'].values 
MU_H = df_operaciones['Flujo_Llegada_por_Min'].values     
CM = df_operaciones['Coste_Rebalanceo_Cm'].iloc[0] 
ALPHA = df_operaciones['Penalizacion_Cuadratica_Alpha'].iloc[0] 

# Asunciones
CAPACIDAD_TOTAL = 24  # Capacidad Total de Slots de la Estación 57 (aproximada)
DURACION_H = 60       # Duración de cada período en minutos
R_MAX_HOURLY = 10     # Máximo de bicicletas a re-balancear por hora
STOCK_INICIAL = 12    # Stock de bicicletas al inicio del día

# Volumen total por hora (en bicicletas)
DEMANDA_H = LAMBDA_H * DURACION_H  # Bikes demanded per hour
OFERTA_H = MU_H * DURACION_H       # Bikes supplied per hour

# Vector de solución inicial: [R_0, ..., R_23, S_0, ..., S_23]
R_initial = np.zeros(N_PERIODS)
S_initial = np.full(N_PERIODS, STOCK_INICIAL) 
X_initial = np.concatenate([R_initial, S_initial])


In [5]:
# --- 2. FUNCIÓN OBJETIVO (NO LINEAL) ---
def objective_function(X):
    R = X[:N_PERIODS] 
    S = X[N_PERIODS:] 

    # Costo Lineal de Re-balanceo (CM * |R_h|)
    cost_rebalance = CM * np.sum(np.abs(R))

    # Outbound loss
    loss_outbound = np.maximum(0, DEMANDA_H - S)

    # Inbound loss: if bikes arrive but no slots available
    loss_inbound = np.maximum(0, OFERTA_H + R - CAPACIDAD_TOTAL)

    # We penalize extreme values to help the optimizer
    cost_nonlinear = ALPHA * (np.sum(loss_outbound**2) + np.sum(loss_inbound**2))
    
    return cost_rebalance + cost_nonlinear

In [ ]:
# --- 3. DEFINICIÓN DE RESTRICCIONES LINEALES ---

# Inventory Balance: S_h + R_h + (O_h - D_h) = S_{h+1}
A_eq = np.zeros((N_PERIODS, 2 * N_PERIODS))
b_eq = np.zeros(N_PERIODS)

for h in range(N_PERIODS):
    # S_h - S_{h+1} + R_h = D_h - O_h
    A_eq[h, N_PERIODS + h] = 1 
    A_eq[h, N_PERIODS + (h + 1) % N_PERIODS] = -1 
    A_eq[h, h] = 1 
    
    # Right side: D_h - O_h
    b_eq[h] = DEMANDA_H[h] - OFERTA_H[h]

# Condición Inicial: S_0 = STOCK_INICIAL
A_stock_init = np.zeros((1, 2 * N_PERIODS))
A_stock_init[0, N_PERIODS] = 1 
b_stock_init = np.array([STOCK_INICIAL])

A_eq_total = np.vstack([A_eq, A_stock_init])
b_eq_total = np.concatenate([b_eq, b_stock_init])

# Límites (Bounds)
bounds_S = [(0, CAPACIDAD_TOTAL)] * N_PERIODS 
bounds_R = [(-R_MAX_HOURLY, R_MAX_HOURLY)] * N_PERIODS
bounds_total = bounds_R + bounds_S

linear_constraint = LinearConstraint(A_eq_total, b_eq_total, b_eq_total)

In [7]:
# --- 4. SOLUCIÓN E INTERPRETACIÓN ---
result = minimize(
    objective_function,
    X_initial,
    method='SLSQP',
    bounds=bounds_total,
    constraints=[linear_constraint],
    options={'ftol': 1e-6, 'disp': False, 'maxiter': 1000}
)
print("\n--- SOLUCIÓN DEL MODELO NO LINEAL (SciPy) ---")

if result.success:
    X_opt = result.x
    R_opt = X_opt[:N_PERIODS] 
    S_opt = X_opt[N_PERIODS:] 

    df_output = df_operaciones[['ID_Periodo_h']].copy()
    
    df_output['Demanda_Salida'] = DEMANDA_H.round(2)
    df_output['Oferta_Llegada'] = OFERTA_H.round(2)
    df_output['Stock_Optimo'] = S_opt.round(2)
    df_output['Rebalanceo_R_h'] = R_opt.round(2)
    
    # Calcular rechazos reales basados en la solución
    df_output['Neto_Rechazo_Llegada'] = np.maximum(0, df_output['Oferta_Llegada'] + R_opt - CAPACIDAD_TOTAL).round(2)
    df_output['Neto_Rechazo_Salida'] = np.maximum(0, df_output['Demanda_Salida'] - S_opt).round(2)
    
    costo_lineal = CM * np.sum(np.abs(R_opt)) 
    costo_nonlinear = objective_function(X_opt) - costo_lineal
    
    print("\n--- Resultados de Optimización Horaria (Estación Crítica) ---")
    
    # Eliminamos numalign y stralign
    print(df_output.head(24).to_string(index=False))
    
    print(f"\nEstado de la Solución: {result.message}")
    print(f"Costo Total Óptimo (Diario): €{result.fun:.2f}")
    print(f"  - Costo Lineal de Re-balanceo (Total): €{costo_lineal:.2f}")
    print(f"  - Costo No Lineal (Penalización Total): €{costo_nonlinear:.2f}")

else:
    print("\n¡ADVERTENCIA! La optimización no convergió.")
    print(result)


--- SOLUCIÓN DEL MODELO NO LINEAL (SciPy) ---

--- Resultados de Optimización Horaria (Estación Crítica) ---
 ID_Periodo_h  Demanda_Salida  Oferta_Llegada  Stock_Optimo  Rebalanceo_R_h  Neto_Rechazo_Llegada  Neto_Rechazo_Salida
            0            7.65            8.03         12.00            0.03                   0.0                 0.00
            1            5.32            5.29         12.42            0.00                   0.0                 0.00
            2            4.84            5.39         12.39            0.00                   0.0                 0.00
            3            2.42            2.48         12.94            0.03                   0.0                 0.00
            4            1.61            1.68         13.03            0.01                   0.0                 0.00
            5            1.45            1.52         13.10            0.01                   0.0                 0.00
            6            1.58            1.61         13.

### Interpretation of Results 

#### 1. Solution Efficiency and Costs
* **Solution Status: `Optimization terminated successfully`**
  The solver (SLSQP) successfully converged to an optimal solution, balancing the cost of moving bicycles with the penalty of rejecting users.
* **Total Daily Cost: `€0.56`**
  * **Linear Rebalancing Cost: `€0.55`**
  * **Non-linear Penalty Cost: `€0.00`**
  The extremely low total cost indicates that the station can be managed almost perfectly. The model found that by spending only **€0.55** on rebalancing, it can completely eliminate user rejections (penalty cost of €0.00).

#### 2. Inventory Stability (Optimal Stock)
* **Stock Management:** The `Stock_Optimo` remains remarkably stable throughout the 24-hour cycle, staying between **11.9** and **13.2** bicycles.
* **Safety Margin:** Since the station capacity is **24 slots**, the model keeps the inventory at approximately **50% capacity**. This is a classic "buffer" strategy: it ensures there are always enough bikes for departures and enough empty docks for arrivals.

#### 3. Rebalancing Strategy ($R_h$)
* **Precision Adjustments:** The model performs very small rebalancing actions (e.g., $+0.03$ or $+0.01$ bikes/min) during most of the day. This suggests that the natural flow of users at Station 57 is relatively balanced.
* **Peak Hour Intervention:** The most significant rebalancing occurs in the evening (Periods 17, 18, and 19):
    * **Period 18:** The model adds bikes ($R_h = 0.60$) to prepare for high evening demand.
    * **Period 19:** The model removes bikes ($R_h = -0.60$) to prevent the station from becoming full as arrivals increase.
* This "just-in-time" rebalancing prevents the station from reaching 0 (empty) or 24 (full).

#### 4. User Experience and Rejections
* **Inbound/Outbound Rejections: `0.00`**
  The primary goal of the model was to minimize the non-linear penalty of rejecting users. The results show **zero rejections** for both people trying to rent a bike and people trying to return one. 
* This demonstrates that with a proactive rebalancing strategy, Station 57 can achieve a **100% service level**.

#### 5. Strategic Conclusion
While **Part A** showed that the network has massive structural imbalances that cannot be fixed just by adding docks, **Part B** proves that at the station level, **logistics (rebalancing) is the key to success.**

* **The Power of Rebalancing:** By making tiny, calculated adjustments to the inventory throughout the day, BiciMad can avoid the "empty station" or "full station" syndrome without expensive infrastructure investments.
* **Operational Recommendation:** Focus on "micro-rebalancing" during the transitions between morning and evening peaks to maintain the inventory near the 50% mark.

### Different Scenarios Analysis

We are going to perform an analysis of different scenarios to evaluate the robustness and scalability of this non-linear model. 

* **Scenario A:** it simulates a big event, such as a concert, a football match or a working day end. Everyone wants to take a bike. We are going to measure if our model is able to replace bikes with the needed frequency. We are going to multiply the demand ($\lambda$) by 2.

* **Scenario B:** it simulates a situationship in which everyone wants to park a bike. For example, the morning when everyone is going to work and arrives to the office. It measures the capacity of releasing parking slots. We are multiplying the supplu ($\mu$) by 2.

* **Scenario C:** it simulates a total chaos day. We are multypling $\lambda$ and $\mu$ by 2.

In [16]:
def objective_function_dynamic(X, demanda_actual, oferta_actual):
    R = X[:N_PERIODS] 
    S = X[N_PERIODS:] 
    
    # Linear Cost
    cost_rebalance = CM * np.sum(np.abs(R))

    # Non - linear penalties
    loss_outbound = np.maximum(0, demanda_actual - S)
    loss_inbound = np.maximum(0, oferta_actual + R - CAPACIDAD_TOTAL)

    cost_nonlinear = ALPHA * (np.sum(loss_outbound**2) + np.sum(loss_inbound**2))
    
    return cost_rebalance + cost_nonlinear

A_eq = np.zeros((N_PERIODS, 2 * N_PERIODS))
for h in range(N_PERIODS):
    # S_h - S_{h+1} + R_h = D - O
    A_eq[h, N_PERIODS + h] = 1 
    A_eq[h, N_PERIODS + (h + 1) % N_PERIODS] = -1 
    A_eq[h, h] = 1 

# Initial stock condition
A_stock_init = np.zeros((1, 2 * N_PERIODS))
A_stock_init[0, N_PERIODS] = 1 
A_eq_total_local = np.vstack([A_eq, A_stock_init])

# We define different scenarios by modifying LAMBDA_H and MU_H
scenarios = {
    "1. High Demand (Salidas x2)": {"lam_factor": 2.0, "mu_factor": 1.0}, 
    "2. High Supply (Llegadas x2)": {"lam_factor": 1.0, "mu_factor": 2.0}, 
    "3. Stress Test (Ambos x2)": {"lam_factor": 2.0, "mu_factor": 2.0}  
}

print("Starting scenario analysis...")

LAMBDA_ORIGINAL = LAMBDA_H.copy()
MU_ORIGINAL = MU_H.copy()

for name, params in scenarios.items():

    print(f"{'='*60}")
    print(f"Processing Scenario: {name}")

    # We change parameters for each scenario
    LAMBDA_H = LAMBDA_ORIGINAL * params["lam_factor"]
    MU_H = MU_ORIGINAL * params["mu_factor"]
    
    DEMANDA_H = LAMBDA_H * DURACION_H 
    OFERTA_H = MU_H * DURACION_H
    
    # We update the right-hand side of the equality constraints as demand and supply change
    b_eq_scenario = np.zeros(N_PERIODS)
    for h in range(N_PERIODS):
        b_eq_scenario[h] = DEMANDA_H[h] - OFERTA_H[h]
        
    # We add the initial stock condition
    b_eq_total_scenario = np.concatenate([b_eq_scenario, b_stock_init])
    
    # We create the new constraint for this scenario
    linear_constraint_scenario = LinearConstraint(A_eq_total, b_eq_total_scenario, b_eq_total_scenario)
    
    # We execute the optimization
    result = minimize(
        objective_function_dynamic, 
        X_initial,
        args = (DEMANDA_H, OFERTA_H),
        method='SLSQP',
        bounds=bounds_total,
        constraints=[linear_constraint_scenario], 
        options={'ftol': 1e-6, 'disp': False, 'maxiter': 1000}
    )
    
    # We show the results
    if result.success:
        X_opt = result.x
        R_opt = X_opt[:N_PERIODS] 
        S_opt = X_opt[N_PERIODS:]
        
        # We compute costs and rejections
        linear_cost = CM * np.sum(np.abs(R_opt)) 
        nonlinear_cost = result.fun - linear_cost
        
        # Total rejections (sum over all hours)
        rechazo_llegada_total = np.sum(np.maximum(0, OFERTA_H + R_opt - CAPACIDAD_TOTAL))
        rechazo_salida_total = np.sum(np.maximum(0, DEMANDA_H - S_opt))
        
        print(f"  - Total Diary Cost: €{result.fun:.2f}")
        print(f"  - Transport Cost: €{costo_lineal:.2f}")
        print(f"  - Penalization: €{costo_nonlinear:.2f}")
        print(f"  - Total Rejections: {rechazo_llegada_total + rechazo_salida_total:.2f} lost users")

LAMBDA_H = LAMBDA_ORIGINAL
MU_H = MU_ORIGINAL
DEMANDA_H = LAMBDA_H * DURACION_H 
OFERTA_H = MU_H * DURACION_H

Starting scenario analysis...
Processing Scenario: 1. High Demand (Salidas x2)
  - Total Diary Cost: €437.10
  - Transport Cost: €4.54
  - Penalization: €65.02
  - Total Rejections: 17.48 lost users
Processing Scenario: 2. High Supply (Llegadas x2)
  - Total Diary Cost: €27.13
  - Transport Cost: €4.54
  - Penalization: €65.02
  - Total Rejections: 0.00 lost users
Processing Scenario: 3. Stress Test (Ambos x2)
  - Total Diary Cost: €69.56
  - Transport Cost: €4.54
  - Penalization: €65.02
  - Total Rejections: 5.68 lost users


### Insights from this Analysis:

* The system shows vulnerability in Scenario A, proving that the current logistic capacity is not enough to cover the entire demand in huge events or peak hours. Increasing the fleet capacity is very important in these cases.

* **Robustness in returns:** in contrast, the system handles properly the High Supply case. The station's capacity buffer combined with standard rebalance is capable of absorbing the increase in returns of bikes.

* **Natural Balance:** Scenario C works better than Scenario A. This suggests that the system naturally self balances when the mobility increases globally (more departures and more arrivals). The high volume of incoming bikes helps satisfying the volume of needed bikes for departures, reducing the need of the rebalancing truck.

### Strategic Conclusions:

According to the system's results, we can conclude that logistics should focus on bringing new bikes to the stations rather than removing them. This is because the system struggles the most when the demand increases, leading to a lot of users' discontent.

We have seen that the performance is clearly worse in the scenario where the demand increases.

We are going to evaluate if setting the starting stock as a decision variable helps the station's performance. 

Right now, the initial stock is set as 12, which may not adapt to different scenarios where more bikes are needed for starting the day.

In [23]:
LAMBDA_ORIGINAL = LAMBDA_H.copy()
MU_ORIGINAL = MU_H.copy()

# We define the equality constraint matrix again without the initial stock condition
A_eq = np.zeros((N_PERIODS, 2 * N_PERIODS))
for h in range(N_PERIODS):
    A_eq[h, N_PERIODS + h] = 1 
    A_eq[h, N_PERIODS + (h + 1) % N_PERIODS] = -1  
    A_eq[h, h] = 1 

scenarios = {
    "1. High Demand (Departures x2)":  {"lam_factor": 2.0, "mu_factor": 1.0},
    "2. High Supply (Arrivals x2)": {"lam_factor": 1.0, "mu_factor": 2.0},
    "3. Stress Test (Both x2)":    {"lam_factor": 2.0, "mu_factor": 2.0}
}

for name, params in scenarios.items():
    
    lambda_scenario = LAMBDA_ORIGINAL * params["lam_factor"]
    mu_scenario = MU_ORIGINAL * params["mu_factor"]
    demand_scen = lambda_scenario * DURACION_H 
    supply_scen = mu_scenario * DURACION_H
    
    # Right-hand side of the equation (b_eq)
    b_eq_scenario = np.zeros(N_PERIODS)
    for h in range(N_PERIODS):
        b_eq_scenario[h] = demand_scen[h] - supply_scen[h]
    
    linear_constraint_scenario = LinearConstraint(A_eq, b_eq_scenario, b_eq_scenario)
    
    # We optimize
    result = minimize(
        objective_function_dynamic, 
        X_initial, # Sigue usando 12 como "semilla", pero puede cambiarlo
        args=(demand_scen, supply_scen),
        method='SLSQP',
        bounds=bounds_total,
        constraints=[linear_constraint_scenario], 
        options={'ftol': 1e-6, 'disp': False}
    )
    
    if result.success:
        X_opt = result.x
        R_opt = X_opt[:N_PERIODS]
        S_opt = X_opt[N_PERIODS:]
        
        initial_stock = S_opt[0]
        integer_initial_stock = int(round(initial_stock))
        
        linear_cost = CM * np.sum(np.abs(R_opt))
        total_cost = result.fun

        rechazo_salida = np.sum(np.maximum(0, demand_scen - S_opt))
        rechazo_llegada = np.sum(np.maximum(0, supply_scen + R_opt - CAPACIDAD_TOTAL))
        total_rechazos = rechazo_salida + rechazo_llegada
        
        print(f"Scenario  {name}")
        print(f"  - Optimal initial stock (rounded): {integer_initial_stock} bikes")
        print(f"  - Total Cost: €{total_cost:.2f}")
        print(f"  - Transport Cost:  €{linear_cost:.2f}")
        print(f"  - Penalization:      €{total_cost - linear_cost:.2f}")
        print(f"  - Users Lost:    {total_rechazos:.2f}")

        # Comparativa rápida si quieres ver diferencia con el modelo anterior
        if name == "1. High Demand (Departures x2)":
            print(f"This used to cost ~437€. The reduced cost is: {437 - total_cost:.2f}€)")
        
        elif name == "3. Stress Test (Both x2)":
            print(f"(This used to cost ~70€. The reduced cost is: {70 - total_cost:.2f}€)")

        print(f"{'='*60}")


# Restaurar
LAMBDA_H = LAMBDA_ORIGINAL
MU_H = MU_ORIGINAL

Scenario  1. High Demand (Departures x2)
  - Optimal initial stock (rounded): 23 bikes
  - Total Cost: €382.97
  - Transport Cost:  €27.13
  - Penalization:      €355.85
  - Users Lost:    14.19
This used to cost ~437€. The reduced cost is: 54.03€)
Scenario  2. High Supply (Arrivals x2)
  - Optimal initial stock (rounded): 13 bikes
  - Total Cost: €27.13
  - Transport Cost:  €27.13
  - Penalization:      €0.00
  - Users Lost:    0.00
Scenario  3. Stress Test (Both x2)
  - Optimal initial stock (rounded): 24 bikes
  - Total Cost: €12.51
  - Transport Cost:  €1.62
  - Penalization:      €10.89
  - Users Lost:    2.39
(This used to cost ~70€. The reduced cost is: 57.49€)


We see that by setting the initial stock as a decision variable the system optimizes its performance in the most critical scenarios. The ones in which the demand increases and the costs were higher. 

In both scenarios in which the demand increased, more than 50€ are saved with this approach.